#### NLP | MVP

# Coronavirus Tweets: Spring 2020<a id='top'></a> 

# 

## **Analysis Goal**  
The client, [name](link), ...
wants to ... and 
requested a model to find a pattern ... in tweets in April. 

Impetus?

**RQ:** ....?

## **Process**
**Data source:** 
Coronavirus COVID-19 Tweets [early](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-early-april) and [late](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-late-april) April


**Models:** 
* [name] model, fit a [model name](#??):
    * eval metric
* [name] model, fit a [model name](#??):
    * eval metric


## **Preliminary Visualizations**
[Figure 1. Name](#??)<br/>
[Figure 1. Name](#??)<br/>

## **Preliminary Conclusions**
The [name] model ... showed....<br/>

The next step .... 

In [ ]:
import glob 
import nltk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import string
pd.set_option('display.max_colwidth', None)
%matplotlib inline
%config InlineBackend.figure_formats = ['retina']  # or svg
sns.set(context='notebook', style='whitegrid')

from itertools import cycle
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.util import ngrams
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 


from cleantext import clean





## 1 | Dataset: Coronavirus Tweets ([early](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-early-april) and [late](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-late-april) April) <a id='1'></a>  

In [ ]:
# # import one csv file to see data
# single_df = pd.read_csv('20200401.csv')

In [ ]:
# single_df.head(2)

In [ ]:
# single_df.info()

In [ ]:
# keep only the following 7 columns on full data import 

#  2   created_at            591480 non-null  object 
#  3   screen_name           591480 non-null  object 
#  4   text                  591480 non-null  object 
#  13  country_code          26040 non-null   object 
#  18  account_lang          0 non-null       float64
#  20  verified              591480 non-null  bool   
#  21  lang                  591480 non-null  object 

In [ ]:
# # read list of April 2020 files, while ommitting columns above

# path = r'/Users/sandraparedes/Documents/GitHub/metis_dsml/05_nlp/g00-nlp-project/data' 
# all_files = glob.glob(path + "/*.csv")

# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0, usecols=[2,3,4,13,18,20,21])
#     li.append(df)
# df = pd.concat(li, axis=0, ignore_index=True)


In [ ]:
# df.head()

In [ ]:
# df.info()

## 2 | Exploratory Data Analysis<a id='2'></a>  

In [ ]:
# df.info()

In [ ]:
# df['country_code'].unique()

In [ ]:
# df['lang'].unique()

In [ ]:
# print('English entries:', (df[df["lang"] == 'en'].count())['lang'])

In [ ]:
# print('US entries:', (df[df["country_code"] == 'US'].count())['country_code'])

In [ ]:
# eng_us_df = df[(df['country_code'] == 'US') &
#                (df['lang'] == 'en')]
# eng_us_df.info()


In [ ]:
# eng_us_df.head(10)

In [ ]:
# eng_us_df['text'].head(10)

### English + US df: 138,789 entries

In [ ]:
# #save cleaned df as tweet_df

# tweet_df = eng_us_df 
# tweet_df.to_pickle('coronavirus_tweets_df.pkl')
# tweet_df.to_csv(r'//Users/sandraparedes/Documents/GitHub/metis_dsml/05_nlp/g00-nlp-project/data/coronavirus_tweets_df.csv', index=False)


In [ ]:
# #read df in
df = pd.read_csv('coronavirus_tweets_df.csv', low_memory=False)
df.head(2)

In [ ]:
df = df.drop(columns=['created_at', 'screen_name', 'country_code', 
                 'account_lang', 'verified', 'lang'])
df.head(2)

### 2a | [start here with nlp](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g1-nlp-overview/NLP_Supervised_Learning.ipynb)<a id='2a'></a>  

### Remove numbers, capital letters, and punctuation

In [ ]:
alphanumeric = lambda x: re.sub('\w*\d\w*',' ', str(x))
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
                          
df['text'] = df.text.map(alphanumeric).map(punc_lower)
df.head(2)



In [ ]:
# remove emojis
df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))


In [ ]:
df.head(2)

In [ ]:
#save clean df

clean_covid_tweets_df = df 
clean_covid_tweets_df.to_pickle('clean_covid_tweets_df.pkl')
clean_covid_tweets_df.to_csv(r'//Users/sandraparedes/Documents/GitHub/metis_dsml/05_nlp/g00-nlp-project/clean_covid_tweets_df.csv', index=False)


In [ ]:
# #read df in
# df = pd.read_csv('clean_covid_tweets_df.csv', low_memory=False)
# df.head(2)

In [ ]:
# Count Vectorizer values - counts of unigrams

X = df.text

cv = CountVectorizer(stop_words='english')

text_cv = cv.fit_transform(X)
# text_cv1 = cv1.transform(X)

text_df_cv = pd.DataFrame(text_cv.toarray(), columns=cv.get_feature_names_out())
text_df_cv


### 3 | [sentiment analysis](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g1-nlp-overview/nlp_unsupervised_nlp_exercises7.ipynb) code blocks<a id='3'></a>  

In [ ]:
analyzer = SentimentIntensityAnalyzer() 
sentiment = analyzer.polarity_scores(df).get('compound')
print('compound', sentiment)

In [ ]:
df['score'] = df.text.map(analyzer.polarity_scores).map(lambda x: x.get('compound'))
df.head()


### 4 | [dim redux](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g2-dimensionality-reduction/nlp_unsupervised_dimensionality_reduction_exercises3.ipynb) and [PCA](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g2-dimensionality-reduction/PCA_digits.ipynb)<a id='4'></a>  

# bookmark Thurs evening
# kernel quits

In [ ]:
pca_5 = PCA(n_components=5)
pca_5.fit(text_df_cv)
# pcafeatures = pca_5.transform(text_df_cv)


In [ ]:
pca_5_components = pca_5.components_


In [ ]:
pc5_positives = []
pc5_positives.append(text_df_cv1.columns[4])
pc5_positives.append(text_df_cv1.columns[5])
pc5_positives

### 4 | [LSA + NMF](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g3-topic-modeling/Topic_Modeling_LSA_NMF.ipynb) [LSA chapter ex](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g3-topic-modeling/nlp_unsupervised_topic_modeling_exercises2.ipynb) 